In [ ]:
# Imports
import os
import numpy as np
import pandas as pd
from scipy.stats import skew, linregress
import matplotlib.pyplot as plt
import seaborn as sns
first_year = 2009
last_year = 2019
years = np.arange(first_year, last_year+1)
# Function to filter and calculate average roughness for elevation bands
def calculate_rms(year, low=100, high=3200, range_step=100):
print(f’Processing {year}’)
dataset = pd.read_csv(f’./yearly/{year}Filtered.csv’) # Load data
dataset = dataset[(dataset[’WGS84_Ellipsoid_Height(m)’] >= low) &
(dataset[’WGS84_Ellipsoid_Height(m)’] <= high)] # Filter to on-ice
rms_data = []
for start in range(low, high, range_step): # Loop through 100 m
end = start + range_step
data = dataset[(dataset[’WGS84_Ellipsoid_Height(m)’] >= start) &
(dataset[’WGS84_Ellipsoid_Height(m)’] < end)]
avg_rms = data[’RMS_Fit(cm)’].mean() if len(data) > 0 else
float(’nan’) # Avg RMS for elevation band
rms_data.append([year, f"{end}", avg_rms])
return rms_data
# Function to filter and calculate skew for elevaiton bands
def calculate_skew(year, low=100, high=3200, range_step=100):
print(f’Calculating skew for {year}’)
dataset = pd.read_csv(f’./yearly/{year}Filtered.csv’) # Load data
dataset = dataset[(dataset[’WGS84_Ellipsoid_Height(m)’] >= low) &
(dataset[’WGS84_Ellipsoid_Height(m)’] <= high)] # Filter to on-ice
skew_data = []
for start in range(low, high, range_step): # Loop through 100 m
end = start + range_step
data = dataset[(dataset[’WGS84_Ellipsoid_Height(m)’] >= start) &
(dataset[’WGS84_Ellipsoid_Height(m)’] < end)]
if len(data) > 0:
hist, _ = np.histogram(data[’RMS_Fit(cm)’], bins=np.linspace(0,
1000, 100)) # Histograms for 100 m
hist_norm = hist / np.sum(hist) # Normalize histogram
skew_in_range = skew(hist_norm) # Calculate skewness
skew_data.append([year, end, skew_in_range])
else:
skew_data.append([year, end, float(’nan’)])
return skew_data
# Calcualte RMS for bands
RMS = [item for year in years for item in calculate_rms(year)]
df_rms = pd.DataFrame(RMS, columns=[’year’, ’elevation’, ’rms’])
df_rms[’elevation’] = pd.to_numeric(df_rms[’elevation’])
# Calculate skew for bands
RMS_skew = [item for year in years for item in calculate_skew(year)]
df_skew = pd.DataFrame(RMS_skew, columns=[’year’, ’elevation’, ’skew’])
df_skew[’elevation’] = pd.to_numeric(df_skew[’elevation’])
# Visualize for RMS above/below 1500 m and all elevations
average_rms_below_1500 = df_rms[df_rms[’elevation’] < 1500].groupby(’year’)
[’rms’].mean().reset_index()
average_rms_above_1500 = df_rms[df_rms[’elevation’] >= 1500].groupby(’year’)
[’rms’].mean().reset_index()
total_rms = df_rms.groupby(’year’)[’rms’].mean().reset_index()
plt.figure(figsize=(10, 6))
plt.scatter(average_rms_below_1500[’year’], average_rms_below_1500[’rms’],
color=’red’, label=’Ablation Zone’)
slope_sub, intercept_sub, _, p_sub, _ = linregress(average_rms_below_1500
[’year’], average_rms_below_1500[’rms’])
plt.plot(average_rms_below_1500[’year’], intercept_sub + slope_sub *
average_rms_below_1500[’year’], color=’red’, linestyle=’--’)
plt.scatter(average_rms_above_1500[’year’], average_rms_above_1500[’rms’],
color=’blue’, label=’Accumulation Zone’)
slope_above, intercept_above, _, p_above, _ = linregress(average_rms_above_1500
[’year’], average_rms_above_1500[’rms’])
plt.plot(average_rms_above_1500[’year’], intercept_above + slope_above *
average_rms_above_1500[’year’], color=’blue’, linestyle=’--’)
plt.scatter(total_rms[’year’], total_rms[’rms’], color=’green’,
label=’All Elevations’)
slope, intercept, _, p, _ = linregress(total_rms[’year’], total_rms[’rms’])
plt.plot(total_rms[’year’], intercept + slope * total_rms[’year’],
color=’green’, linestyle=’--’)
plt.text(0.05, 0.79, f’\nslope={slope_sub:.2f}, p={p_sub:.2f}’,
transform=plt.gca().transAxes, color=’red’, verticalalignment=’top’)
plt.text(0.05, 0.48, f’\nslope={slope:.2f}, p={p:.2f}’,
transform=plt.gca().transAxes, color=’green’, verticalalignment=’top’)
plt.text(0.05, 0.22, f’\nslope={slope_above:.2f}, p={p_above:.2f}’,
transform=plt.gca().transAxes, color=’blue’, verticalalignment=’top’)
plt.ylim(0,120)
plt.xlabel(’Year’, fontsize=14)
plt.ylabel(’Average Surface Roughness [cm]’, fontsize=14)
plt.title(’Average Surface Roughness Over Time’, fontsize=16)
plt.legend(loc=’upper right’, fontsize=11)
plt.grid(True)
plt.show()
# Visualize rate of change in roughness for 100 meter bands
elevations = sorted(df_rms[’elevation’].unique())
elevation_vals, slope_vals = [], []
plt.figure(figsize=(10, 6))
for elev in elevations:
data_in = df_rms[df_rms[’elevation’] == elev]
slope, *_ = linregress(data_in[’year’], data_in[’rms’])
elevation_vals.append(elev)
slope_vals.append(slope)
plt.plot(elevation_vals, slope_vals, linestyle=’--’, marker=’o’, color=’b’)
plt.xlabel(’Elevation [m]’, fontsize=14)
plt.ylabel(’Rate of Change in Roughness [cm/yr]’, fontsize=14)
plt.title(’Rate of Change in Surface Roughness with Elevation’, fontsize=16)
plt.grid(True)
plt.show()
# Visualize 100 m avg vs elevation
year_colors = {
2009: ’red’, 2010: ’orange’, 2011: ’yellow’, 2012: ’lightgreen’, 2013:
’green’, 2014: ’lightblue’, 2015: ’blue’, 2016: ’indigo’, 2017: ’violet’,
2018: ’gray’, 2019: ’black’
}
plt.figure(figsize=(10, 6))
for year, group in df_rms.groupby(’year’):
plt.plot(group[’elevation’], group[’rms’], label=str(year),
color=year_colors.get(year, ’black’), linestyle=’--’)
plt.xlabel(’Elevation [m]’, fontsize=16)
plt.ylabel(’Average Roughness [cm]’, fontsize=16)
plt.title(’Yearly Roughness vs. Elevation’, fontsize=16)
plt.legend(fontsize=13)
plt.grid(True)
plt.show()
# Calculate avg skewnesss for 100m bands
RMS_skew = []
RMS_skew_sub1500 = []
RMS_skew_above1500 = []
for year in years:
print(f’Processing year {year}’)
dataset = pd.read_csv(f’./yearly/{year}Filtered.csv’)
dataset = dataset[(dataset[’WGS84_Ellipsoid_Height(m)’] >= 100) &
(dataset[’WGS84_Ellipsoid_Height(m)’] <= 3200)]
# Calc normalized histogram and skew
roughness_bins = np.linspace(0, 1000, 100)
hist, _ = np.histogram(dataset[’RMS_Fit(cm)’], bins=roughness_bins)
hist_norm = hist / np.sum(hist) # Normalize histogram
skew_total = skew(hist_norm)
RMS_skew.append([year, skew_total])
# Filter and calculate skewness <1500 m
dataset_sub1500 = dataset[(dataset[’WGS84_Ellipsoid_Height(m)’] >= 100)
& (dataset[’WGS84_Ellipsoid_Height(m)’] <= 1500)]
hist_sub1500, _ = np.histogram(dataset_sub1500[’RMS_Fit(cm)’],
bins=roughness_bins)
hist_norm_sub1500 = hist_sub1500 / np.sum(hist_sub1500) # Normalize
skew_sub1500 = skew(hist_norm_sub1500)
RMS_skew_sub1500.append([year, skew_sub1500])
# Filter and calculate skew for elevations above 1500m
dataset_above1500 = dataset[(dataset[’WGS84_Ellipsoid_Height(m)’] >
1500) & (dataset[’WGS84_Ellipsoid_Height(m)’] <= 3200)]
hist_above1500, _ = np.histogram(dataset_above1500[’RMS_Fit(cm)’],
bins=roughness_bins)
hist_norm_above1500 = hist_above1500 / np.sum(hist_above1500)
skew_above1500 = skew(hist_norm_above1500)
RMS_skew_above1500.append([year, skew_above1500])
# Visualize skew above/below 1500 m
df_total = pd.DataFrame(RMS_skew, columns=[’year’, ’skew’])
df_sub1500 = pd.DataFrame(RMS_skew_sub1500, columns=[’year’, ’skew’])
df_above1500 = pd.DataFrame(RMS_skew_above1500, columns=[’year’, ’skew’])
plt.figure(figsize=(10, 6))
plt.scatter(df_sub1500[’year’], df_sub1500[’skew’], label=’Ablation Zone’,
marker=’o’, color=’red’)
slope_sub, intercept_sub, _, p_sub, _ = linregress(df_sub1500[’year’],
df_sub1500[’skew’])
regress_line_sub = slope_sub * df_sub1500[’year’] + intercept_sub
plt.plot(df_sub1500[’year’], regress_line_sub, linestyle=’--’, color=’red’)
plt.scatter(df_above1500[’year’], df_above1500[’skew’], label=’Accumulation
Zone’, marker=’o’, color=’blue’)
slope_above, intercept_above, _, p_above, _ = linregress(df_above1500[’year’],
df_above1500[’skew’])
regress_line_above = slope_above * df_above1500[’year’] + intercept_above
plt.plot(df_above1500[’year’], regress_line_above, linestyle=’--’,
color=’blue’)
plt.scatter(df_total[’year’], df_total[’skew’], label=’All Elevations’,
marker=’o’, color=’green’)
slope, intercept, _, p, _ = linregress(df_total[’year’], df_total[’skew’])
regress_line = slope * df_total[’year’] + intercept
plt.plot(df_total[’year’], regress_line, linestyle=’--’, color=’green’)
plt.text(0.05, 0.88, f’\nslope={slope_above:.2f}, p={p_above:.2f}’,
transform=plt.gca().transAxes, color=’blue’, verticalalignment=’top’)
plt.text(0.05, 0.73, f’\nslope={slope:.2f}, p={p:.2f}’, transform=
plt.gca().transAxes, color=’green’, verticalalignment=’top’)
plt.text(0.05, 0.3, f’\nslope={slope_sub:.2f}, p={p_sub:.2f}’, transform=
plt.gca().transAxes, color=’red’, verticalalignment=’top’)
plt.xlabel(’Year’, fontsize=14)
plt.ylabel(’Skew’, fontsize=14)
plt.title(’Normalized Skew Over Time’, fontsize=16)
plt.legend(fontsize=12)
plt.xticks(years, rotation=45)
plt.tight_layout()
plt.grid(True)
plt.show()
plt.figure(figsize=(10, 6))
below_1500 = df_skew[df_skew[’elevation’] < 1600]
elevation_colors = {
200:’brown’, 300:’red’, 400:’orangered’, 500:’orange’, 600:’gold’,
700:’lime’, 800:’green’, 900:’lightseagreen’, 1000:’blue’, 1100:
’navy’, 1200:’mediumorchid’, 1300:’deeppink’, 1400:’gray’, 1500:’black’,
}
elevation_ranges = {
(100, 200): elevation_colors[200],(200, 300): elevation_colors[300],
(300, 400): elevation_colors[400],(400, 500): elevation_colors[500],
(500, 600): elevation_colors[600],(600, 700): elevation_colors[700],
(700, 800): elevation_colors[800],(800, 900): elevation_colors[900],
(900, 1000): elevation_colors[1000], (1000, 1100): elevation_colors[1100],
(1100, 1200): elevation_colors[1200],(1200, 1300): elevation_colors[1300],
(1300, 1400): elevation_colors[1400],(1400, 1500): elevation_colors[1500]
}
legend_labels = set()
sorted_elevations = sorted(below_1500[’elevation’].unique())
for elevation in sorted_elevations:
data_at_elevation = below_1500[below_1500[’elevation’]==
elevation].dropna()
if not data_at_elevation.empty:
color = elevation_colors.get(elevation, ’gray’)
plt.scatter(data_at_elevation[’year’], data_at_elevation[’skew’],
alpha=0.7, color=color)
# Get elevation range for the legend
for (low, high), col in elevation_ranges.items():
if low < elevation <= high:
legend_label = f’{low}-{high} m’
if legend_label not in legend_labels:
plt.scatter([], [], color=col, label=legend_label)
legend_labels.add(legend_label)
# Linear regression line for skew values
slope, intercept, *_ = linregress(data_at_elevation[’year’],
data_at_elevation[’skew’])
regress_line = slope * data_at_elevation[’year’] + intercept
plt.plot(data_at_elevation[’year’], regress_line, linestyle=’--’,
color=color)
plt.xlabel(’Year’, fontsize=14)
plt.ylabel(’Skew’, fontsize=14)
plt.title(’Normalized Skew of Ablation Zone Roughness Over Time’,
fontsize=16)
plt.legend(loc=’center left’, bbox_to_anchor=(1, 0.5), fontsize=11)
plt.xticks(rotation=0)
plt.tight_layout()
plt.grid(True)
plt.show()